# Deep Sonic
### ___Chris Pagolu, Joshua JJ Wonder___

[DeepSonic](https://magenta.tensorflow.org/) is a fully open source Deep Learning music experiment that is capable of synthesizing, generating, remixing, modifying music, all using the power of AI. Thanks to powerful open-source libraries such as Magenta by Tensorflow/Google and Jukebox by OpenAI, we were able to create a multi-functional AI Audio Engineer. 

Note: This notebook runs all code natively. No cloud service is required unless you do not have a dedicated Nvidia GPU.

# Basic Hardware Requirements and Recommendations

The DeepSonic Experiment requires considerably powerful hardware. 

An NVIDIA Geforce RTX 2000 (Turing) Series GPU with atleast 8GB of VRAM is required, at the least. A cloud-based NVIDIA Tesla or a server NVIDIA Quadro GPU with atleast 16GB VRAM is recommended, while a supercomputer will perform best, depending on the task. There are no explicit CPU requirements for DeepSonic, however, an AMD Ryzen 3 3200 or Intel Core i3 8100 (or higher) is recommended. The more powerful, the better.

# How to get started?

To get started with DeepSonic, all you have to do is install Magenta and Jukebox from their official GitHub repositories.

# Quick Install Guide:

Run the follow code in your shell (taken from the official Magenta and Jukebox wiki) to install the required tools. We recommend using a Debian-based operating system. The Windows Subsystem for Linux (WSL2) and Windows didn't appear to work at the time of our testing, possibly due to early support for Cuda on WSL. Also note: root privileges are required for installing audio libraries.

```bash
# Required commands:

sudo apt-get update && sudo apt-get install build-essential libasound2-dev libjack-dev portaudio19-dev
curl https://raw.githubusercontent.com/tensorflow/magenta/main/magenta/tools/magenta-install.sh > /tmp/magenta-install.sh
bash /tmp/magenta-install.sh
conda create --name jukebox python=3.7.5
conda activate jukebox
conda install mpi4py=3.0.3 # if this fails, try: pip install mpi4py==3.0.3
conda install pytorch=1.4 torchvision=0.5 cudatoolkit=10.0 -c pytorch
git clone https://github.com/openai/jukebox.git
cd jukebox
pip install -r requirements.txt
pip install -e .
conda install av=7.0.01 -c conda-forge 
pip install ./tensorboardX
 
# Following two commands are optional: Apex for faster training with fused_adam 

conda install pytorch=1.1 torchvision=0.3 cudatoolkit=10.0 -c pytorch
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
```

In [1]:
print('Importing Modules...\n')
#basic libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
%matplotlib inline

#magenta libraries
from magenta.models.nsynth import utils
from magenta.models.nsynth.wavenet import fastgen
from note_seq.notebook_utils import colab_play as play


#jukebox libraries

import jukebox
import torch as t
import librosa
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
rank, local_rank, device = setup_dist_from_mpi()

get_name = lambda f: os.path.splitext(os.path.basename(f))[0]

print('Sucess!! Environment is now setup.')

Importing Modules...

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Using cuda True
Sucess!! Environment is now setup.


In [2]:
!nvidia-smi #checks if cuda and nvidia drivers are working properly

Sun Aug 22 15:07:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:09:00.0  On |                  N/A |
| 25%   49C    P5    16W / 215W |   2234MiB /  7981MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# DeepSynth
### __Adapted from the [EZSynth Experiment](https://colab.research.google.com/notebooks/magenta/nsynth/nsynth.ipynb) by Jesse Engel, Cinjon Resnick, Adam Roberts, Sander Dieleman, Douglas Eck, Karen Simonyan, Mohammad Norouzi__
Neural Audio Synthesis of Musical Notes with WaveNet Autoencoders

### Additional Resources (as provided in the [EZSynth Notebook](https://colab.research.google.com/notebooks/magenta/nsynth/nsynth.ipynb)):
* [Nat and Friends "Behind the scenes"](https://www.youtube.com/watch?v=BOoSy-Pg8is)
* [Original Blog Post](https://magenta.tensorflow.org/nsynth)
* [NSynth Instrument](https://magenta.tensorflow.org/nsynth-instrument)
* [Jupyter Notebook Tutorial](https://magenta.tensorflow.org/nsynth-fastgen)
* [ArXiv Paper](https://arxiv.org/abs/1704.01279)
* [Github Code](https://github.com/tensorflow/magenta/tree/main/magenta/models/nsynth)

There are two pretrained models to choose from (thanks, Google :)): one trained on the individual instrument notes of the [NSynth Dataset](https://magenta.tensorflow.org/datasets/nsynth) ("Instruments"), and another trained on a variety of voices in the wild for an art project ("Voices", mixture of singing and speaking). The Instruments model was trained on a larger quantity of data, so tends to generalize a bit better. Neither reconstructs audio perfectly, but both add their own unique character to sounds. 

In [ ]:
#Choose a Model { vertical-output: true, run: "auto" }
Model = "Instruments" #@param ["Instruments", "Voices"] {type:"string"}
ckpts = {'Instruments': 'wavenet-ckpt/model.ckpt-200000',
         'Voices': 'wavenet-voice-ckpt/model.ckpt-200000'}

ckpt_path = ckpts[Model]
print('Using model pretrained on %s.' % Model)

# Use local audio files

In the next section, you may choose to specify which audio file you want to use for the audio synthesis. Note: the larger your audio file, the longer it'll take to encode and the longer it'll take to synthesize the audio, depending on how powerful your GPU is.

In [ ]:
#Set Sound Length (in Seconds) { vertical-output: true, run: "auto" }
Length = 60.0 #set the length of your synthesized audio
SR = 16000
SAMPLE_LENGTH = int(SR * Length) #audio length

In [ ]:
#Upload sound files (.wav, .mp3)

try:
  file_list, audio_list = [], [] #creates numpy arrays for file and audio lists
  fname="test.mp3" # name of the audio file
  audio = utils.load_audio(fname, sample_length=SAMPLE_LENGTH, sr=SR)  #loads audio file for magenta to process
  file_list.append(fname)
  audio_list.append(audio)
  names = [get_name(f) for f in file_list]
  # Pad and peak normalize
  for i in range(len(audio_list)):
    audio_list[i] = audio_list[i] / np.abs(audio_list[i]).max()

    if len(audio_list[i]) < SAMPLE_LENGTH:
      padding = SAMPLE_LENGTH - len(audio_list[i])
      audio_list[i] = np.pad(audio_list[i], (0, padding), 'constant')

  audio_list = np.array(audio_list)
except Exception as e:
  print("Error encountered. Sure the file is .wav or .mp3? Does your GPU have enough memory left?")
  print(e)

The below code may take some time, depending on your GPU.

In [ ]:
#Generate Encodings
audio = np.array(audio_list)
z = fastgen.encode(audio, ckpt_path, SAMPLE_LENGTH)
print('Encoded %d files' % z.shape[0])


# Start with reconstructions
z_list = [z_ for z_ in z]
name_list = ['recon_' + name_ for name_ in names]

# Add all the mean interpolations
n = len(names)
for i in range(n - 1):
  for j in range(i + 1, n):
    new_z = (z[i] + z[j]) / 2.0
    new_name = 'interp_' + names[i] + '_X_'+ names[j]
    z_list.append(new_z)
    name_list.append(new_name)

print("%d total: %d reconstructions and %d interpolations" % (len(name_list), n, len(name_list) - n))

# Final Step: Synthesize

With a GPU, this should take about 4 minutes per 1 second of audio per a batch. Approximate time required for a 60 second song (~1,000,000 interpolations): 8-12 hours on a GeForce GPU and 2-8 hours on a Quadro or Tesla GPU. After that, your synthesized audio will appear in the same directory as this notebook (can be found as `recon_<name of audio>.mp3` or `recon_<name of audio>.wav`). )

In [ ]:
#Synthesize Interpolations
print('Total Iterations to Complete: %d\n' % SAMPLE_LENGTH)

encodings = np.array(z_list)
save_paths = [name + '.wav' for name in name_list]
fastgen.synthesize(encodings,
                   save_paths=save_paths,
                   checkpoint_path=ckpt_path,
                   samples_per_save=int(SAMPLE_LENGTH / 10))

# DeepLyrics
## Using the power of Jukebox, an open-source Deep Learning music generation library, developed by OpenAI to generate music from nothing but lyrics.
### Adapted from the Jukebox Colab Notebook

Note: We highly recommend that you follow the recommended hardware specifications specified earlier for this particular segment. We also highly recommend that you run this on a GPU with atleast 16GB of memory.

# Sample from 1B or 5B model
The 5B model is more robust and will provide better results compared to the 1B model. However, the 1B model is signifantly faster and less resource intensive. Use it if you have less than 16GB of GPU memory on your system.

In [ ]:
model = "1b_lyrics" # Change this to "5b_lyrics" if you choose to use the 5B model
hps = Hyperparams() #load hyperparams
hps.sr = 44100 #sample rate
hps.n_samples = 3 if model=='5b_lyrics' else 3 #number of samples to generate
hps.name = 'samples'
chunk_size = 16 if model=="5b_lyrics" else 32
max_batch_size = 3 if model=="5b_lyrics" else 16
hps.levels = 3
hps.hop_fraction = [.5,.5,.125]

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 1048576)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)

Specify your choice of artist, genre, lyrics, and length of musical sample. 

In [ ]:
sample_length_in_seconds = 60          # Full length of musical sample to generate - we find songs in the 1 to 4 minute
                                       # range work well, with generation time proportional to sample length.  
                                       # This total length affects how quickly the model 
                                       # progresses through lyrics (model also generates differently
                                       # depending on if it thinks it's in the beginning, middle, or end of sample)

hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
assert hps.sample_length >= top_prior.n_ctx*top_prior.raw_to_tokens, f'Please choose a larger sampling rate'

In [ ]:
#We chose to work with Eminem's voice with lyrics from "Paid my Dues" by NF

metas = [dict(artist = "Eminem",
            genre = "Hip Hop",
            total_length = hps.sample_length,
            offset = 0,
            lyrics = """II spit it with ease, so leave it to me
            You doubt it but you better believe
            I'm on a rampage hit 'em with the record release
            Dependin' the week, I'm prolly gonna have to achieve another goal
            Let me go when I'm over the beat
            I go into beast mode like I'm ready to feast
            I'm fed up with these thieves tryna get me to bleed
            They wanna see me take an L? (yup, see what I mean)
            How many records I gotta give you to get with the program?
            Taken for granted I'm 'bout to give you the whole plan
            Open your mind up and take a look at the blueprint
            Debate if you gotta, but gotta hold it with both hands
            To pick up the bars you gotta be smart
            You really gotta dig in your heart if you wanna get to the root of an issue
            Pursuin' the mental can be dark and be difficult
            But the payoff at the end of it, can help you to get through it, hey
            """,
            ),
          ] * hps.n_samples
labels = [None, None, top_prior.labeller.get_batch_labels(metas, 'cuda')]

Optionally adjust the sampling temperature (set it around 1 for the best results).  

In [ ]:
sampling_temperature = .98

lower_batch_size = 16
max_batch_size = 3 if model == "5b_lyrics" else 16
lower_level_chunk_size = 32
chunk_size = 16 if model == "5b_lyrics" else 32
sampling_kwargs = [dict(temp=.99, fp16=True, max_batch_size=lower_batch_size,
                        chunk_size=lower_level_chunk_size),
                    dict(temp=0.99, fp16=True, max_batch_size=lower_batch_size,
                         chunk_size=lower_level_chunk_size),
                    dict(temp=sampling_temperature, fp16=True, 
                         max_batch_size=max_batch_size, chunk_size=chunk_size)]

Now we're ready to sample from the model. We'll generate the top level (2) first, followed by the first upsampling (level 1), and the second upsampling (0).  If you are using a local machine, you can also load all models directly with make_models, and then use sample.py's ancestral_sampling to put this all in one step.

After each level, we decode to raw audio and save the audio files.   

This next cell will take a while (approximately 10 minutes per 20 seconds of music sample), similar to synthesizing audio as we demonstrated earlier.

Approximate time required for a 60 second song (~1,000,000 interpolations): 30 mins-1 hour on a GeForce RTX GPU and 5-10 minutes on a Quadro or Tesla GPU. These audio files are compressed and will be of lower audio quality than the original. You may find these at {hps.name}/level_2/.

In [ ]:
zs = [t.zeros(hps.n_samples,0,dtype=t.long, device='cuda') for _ in range(len(priors))]
zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)

In [ ]:
Audio(f'{hps.name}/level_2/item_0.wav') #if you want to hear it directly from Jupyter or Colab

# Upsampling

The following code block will allow you to upsample your previously generated audio using Neural Networks. This process is GPU dependant and will take a long time to complete if you do not meet the recommended hardware requirements. With a GPU, this should take about 4 minutes per 1 second of audio per a batch. Approximate time required for a 60 second song (~800,000 interpolations): 8-12 hours on a GeForce GPU and 2-8 hours on a Quadro or Tesla GPU. The resultant file will be available at 

In [ ]:
# Set this False if you are on a local machine that has enough memory (this allows you to do the
# lyrics alignment visualization during the upsampling stage). For a hosted runtime, 
# we'll need to go ahead and delete the top_prior if you are using the 5b_lyrics model.
if True:
  del top_prior
  empty_cache()
  top_prior=None
upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]
labels[:2] = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers]

In [ ]:
zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps) #Note: This is the code that upsamples the previously 
                                                                          #generated low-quality audio file

In [ ]:
Audio(f'{hps.name}/level_0/item_0.wav') #if you want to hear it directly from Jupyter or Colab

In [ ]:
del upsamplers
empty_cache() #clears stored cache from all the processing

ModuleNotFoundError: No module named 'openai'